<a href="https://colab.research.google.com/github/Dewarya/diabetic-retinopathy-detection_ML_DL_CNN/blob/main/Diabetic_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!mkdir ~/.kaggle

In [3]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sovitrath/diabetic-retinopathy-224x224-2019-data

In [6]:
import zipfile
zip_ref = zipfile.ZipFile('/content/diabetic-retinopathy-224x224-2019-data.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from keras.preprocessing import image


In [8]:
df=pd.read_csv("/content/train.csv")

In [ ]:
df

In [ ]:
# Add an additional column, mapping to the type
df = pd.read_csv(r'/content/train.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


df['binary_type'] =  df['diagnosis'].map(diagnosis_dict_binary.get)
df['type'] = df['diagnosis'].map(diagnosis_dict.get)
df.head()

In [ ]:
# Split into stratified train, val, and test sets
train_intermediate, val = train_test_split(df, test_size=0.15, stratify=df['type'])
train, test = train_test_split(train_intermediate, test_size=0.15 / (1 - 0.15), stratify=train_intermediate['type'])

print(train['type'].value_counts(), '\n')
print(test['type'].value_counts(), '\n')
print(val['type'].value_counts(), '\n')

In [12]:
# Create working directories for train/val/test
base_dir = Path('/content/DL_Diabetic_Retinopathy')
train_dir = base_dir / 'train'
val_dir = base_dir / 'val'
test_dir = base_dir / 'test'

# Create directories
for directory in [base_dir, train_dir, val_dir, test_dir]:
    directory.mkdir(parents=True, exist_ok=True)


In [13]:
# Copy images to respective working directory
src_dir = Path('/content/colored_images')
for index, row in train.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['binary_type']
    id_code = row['id_code'] + ".png"
    srcfile = src_dir / diagnosis / id_code
    dstfile = train_dir / binary_diagnosis / id_code
    dstfile.parent.mkdir(parents=True, exist_ok=True)  # Ensure parent directory exists
    with open(srcfile, 'rb') as src, open(dstfile, 'wb') as dst:
        dst.write(src.read())

for index, row in val.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['binary_type']
    id_code = row['id_code'] + ".png"
    srcfile = src_dir / diagnosis / id_code
    dstfile = val_dir / binary_diagnosis / id_code
    dstfile.parent.mkdir(parents=True, exist_ok=True)  # Ensure parent directory exists
    with open(srcfile, 'rb') as src, open(dstfile, 'wb') as dst:
        dst.write(src.read())

for index, row in test.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['binary_type']
    id_code = row['id_code'] + ".png"
    srcfile = src_dir / diagnosis / id_code
    dstfile = test_dir / binary_diagnosis / id_code
    dstfile.parent.mkdir(parents=True, exist_ok=True)  # Ensure parent directory exists
    with open(srcfile, 'rb') as src, open(dstfile, 'wb') as dst:
        dst.write(src.read())

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_dir, target_size=(224, 224), batch_size=32)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir, target_size=(224, 224), batch_size=32)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir, target_size=(224, 224), batch_size=32)


In [ ]:
# Building the model

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), padding="valid", input_shape=(224,224,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(16, (3,3), padding="valid", activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(32, (4,4), padding="valid", activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=val_batches)

loss, acc = model.evaluate_generator(test_batches, verbose=1)
print("Accuracy: ", acc)


In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
# Make predictions
predictions = model.predict(test_batches)
y_true = test_batches.classes
y_pred = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [19]:
print("Class indices:", train_batches.class_indices)

Class indices: {'DR': 0, 'No_DR': 1}


In [24]:
# Load the image you want to test
test_image_path = "/content/25046-retinal-imaging.png"
test_image = image.load_img(test_image_path, target_size=(224, 224))
test_image_array = image.img_to_array(test_image)
test_image_array = np.expand_dims(test_image_array, axis=0)

# Preprocess the image
preprocessed_test_image = test_image_array / 255.0  # Normalize pixel values

# Use the trained model to make predictions
predictions = model.predict(preprocessed_test_image)
predicted_class_index = np.argmax(predictions)

# Reverse the class indices dictionary
class_labels = {v: k for k, v in train_batches.class_indices.items()}

# Get the predicted class label using the reversed dictionary
predicted_class = class_labels[predicted_class_index]

# Interpret the predictions
print("Predicted class:", predicted_class)
print("Predicted probabilities:", predictions)

1/1 [==============================] - 0s 268ms/step
Predicted class: No_DR
Predicted probabilities: [[3.3655514e-09 1.0000000e+00]]


In [25]:
# Load the image you want to test
test_image_path = "/content/0083ee8054ee.png"
test_image = image.load_img(test_image_path, target_size=(224, 224))
test_image_array = image.img_to_array(test_image)
test_image_array = np.expand_dims(test_image_array, axis=0)

# Preprocess the image
preprocessed_test_image = test_image_array / 255.0  # Normalize pixel values

# Use the trained model to make predictions
predictions = model.predict(preprocessed_test_image)
predicted_class_index = np.argmax(predictions)

# Reverse the class indices dictionary
class_labels = {v: k for k, v in train_batches.class_indices.items()}

# Get the predicted class label using the reversed dictionary
predicted_class = class_labels[predicted_class_index]

# Interpret the predictions
print("Predicted class:", predicted_class)
print("Predicted probabilities:", predictions)

1/1 [==============================] - 0s 33ms/step
Predicted class: DR
Predicted probabilities: [[9.9999988e-01 8.9387775e-08]]
